In [1]:
import os
import re
import glob
import datetime
import warnings

import numpy as np
import pandas as pd
import xarray as xr

In [2]:
## This section caches all MODIS MCD43A4 file names in the format listed below, where '?' is a wildcard
try:
    with open('/g/data/oe9/modis_tile_file_list.txt') as f:
        reflectance_file_cache = sorted(l.strip() for l in f.readlines())
except:
    reflectance_file_cache =  sorted(glob.glob(
        '/g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/'
        '????.??.??/MCD43A4.A???????.h??v??.006.*.hdf'
    ))
    with open('/g/data/oe9/modis_tile_file_list.txt', 'w') as f:
        f.writelines(l + '\n' for l in reflectance_file_cache)
len(reflectance_file_cache)

504980

In [17]:
def add_tile_coords(tile, dataset):
    scale = 1111950.5196669996

    # regex to match string
    regex = re.compile('h\d+v\d+')
    matches = regex.findall(tile)

    # extract values from string
    extract = re.compile('\d+')
    h, v = extract.findall(matches[0])
    h = int(h)
    v = int(v)

    # calculate start and end values
    x_start = scale * (h - 18)
    x_end = scale * (h - 17)

    y_start = -scale * (v - 9)
    y_end = -scale * (v - 8)

    dataset['x'] = xr.IndexVariable('x', np.linspace(x_start, x_end, 2400))
    dataset['y'] = xr.IndexVariable('y', np.linspace(y_start, y_end, 2400))
    return dataset


In [18]:
## Rename the bands to something useful
modis_band_map = {
    'Nadir_Reflectance_Band1': 'red_630_690',
    'Nadir_Reflectance_Band2': 'nir1_780_900',
    'Nadir_Reflectance_Band3': 'blue_450_520',
    'Nadir_Reflectance_Band4': 'green_530_610',
    'Nadir_Reflectance_Band5': 'nir2_1230_1250',
    'Nadir_Reflectance_Band6': 'swir1_1550_1750',
    'Nadir_Reflectance_Band7': 'swir2_2090_2350',
}

## define function 'get_reflectance' whose input is tile ID 'h__v__'
## This function searches the cached list for filenames which contain tile 'h__v__
## outputs data as a concatenated dataset with dimensions, time, x ,y 
def get_reflectance(tile):
    assert re.match(r'h\d\dv\d\d', tile), 'tile must be string "h__v__"'
    files = [f for f in reflectance_file_cache if tile in os.path.basename(f)]
    pattern = re.compile(r'MCD43A4.A(?P<year>\d{4})(?P<day>\d{3}).h\d\dv\d\d.006.\d+.hdf')
    dates, parts = [], []
    for f in files:
        assert os.path.isfile(f)
        parts.append(xr.open_dataset(f, chunks=2400,autoclose=True))
        year, day = pattern.match(os.path.basename(f)).groups()
        dates.append(datetime.date(int(year), 1, 1) +
                     datetime.timedelta(days=int(day) - 1))

    dates = pd.to_datetime(dates)
    dates.name = 'time'

    ds = xr.concat(parts, dates)
    out = xr.Dataset()
    for i in map(str, range(1, 8)):
        key = 'Nadir_Reflectance_Band' + i
        data_ok = ds['BRDF_Albedo_Band_Mandatory_Quality_Band' + i] == 0
        out[modis_band_map[key]] = ds[key].where(data_ok).astype('f4')

    out.rename({'YDim:MOD_Grid_BRDF': 'y',
                'XDim:MOD_Grid_BRDF': 'x'}, inplace=True)
    out.time.encoding.update(dict(
        units='days since 1900-01-01', calendar='gregorian', dtype='i4'))
    for var in out.data_vars:
        out[var].encoding.update(dict(
            shuffle=True, zlib=True, chunks=dict(x=240, y=240, time=6), dtype='f4',
        ))
    return add_tile_coords(tile, out)

In [4]:
%%time
ds = get_reflectance('h30v12')
ds

CPU times: user 43.6 s, sys: 43.2 s, total: 1min 26s
Wall time: 4min 30s


In [21]:
ds

<xarray.Dataset>
Dimensions:          (time: 1643, x: 2400, y: 2400)
Coordinates:
  * time             (time) datetime64[ns] 2000-02-26 2000-03-01 2000-03-05 ...
  * x                (x) float64 1.334e+07 1.334e+07 1.334e+07 1.334e+07 ...
  * y                (y) float64 -3.336e+06 -3.336e+06 -3.337e+06 -3.337e+06 ...
Data variables:
    red_630_690      (time, y, x) float32 dask.array<shape=(1643, 2400, 2400), chunksize=(1, 2400, 2400)>
    nir1_780_900     (time, y, x) float32 dask.array<shape=(1643, 2400, 2400), chunksize=(1, 2400, 2400)>
    blue_450_520     (time, y, x) float32 dask.array<shape=(1643, 2400, 2400), chunksize=(1, 2400, 2400)>
    green_530_610    (time, y, x) float32 dask.array<shape=(1643, 2400, 2400), chunksize=(1, 2400, 2400)>
    nir2_1230_1250   (time, y, x) float32 dask.array<shape=(1643, 2400, 2400), chunksize=(1, 2400, 2400)>
    swir1_1550_1750  (time, y, x) float32 dask.array<shape=(1643, 2400, 2400), chunksize=(1, 2400, 2400)>
    swir2_2090_2350  (time, 

In [14]:
ds.nbytes/10**9

264.983053144

In [20]:
ds.x

<xarray.DataArray 'x' (x: 2400)>
array([ 13343406.236004,  13343869.741848,  13344333.247692, ...,
        14454429.743983,  14454893.249827,  14455356.755671])
Coordinates:
  * x        (x) float64 1.334e+07 1.334e+07 1.334e+07 1.334e+07 1.335e+07 ...

In [31]:
## netcdf creation
## save w/chunks on disk
## 



In [30]:
# for var in ds.data_vars:
#     print(repr(var), type(var))
#     print(ds[var])
#     break

'red_630_690' <class 'str'>
<xarray.DataArray 'red_630_690' (time: 1643, y: 2400, x: 2400)>
dask.array<shape=(1643, 2400, 2400), dtype=float32, chunksize=(1, 2400, 2400)>
Coordinates:
  * time     (time) datetime64[ns] 2000-02-26 2000-03-01 2000-03-05 ...
  * x        (x) float64 1.334e+07 1.334e+07 1.334e+07 1.334e+07 1.335e+07 ...
  * y        (y) float64 -3.336e+06 -3.336e+06 -3.337e+06 -3.337e+06 ...


NameError: name 'breaj' is not defined